In [1]:
import mne
from sklearn.decomposition import FastICA, PCA
import warnings
from mne.preprocessing import ICA
warnings.filterwarnings('ignore')

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


In [3]:
#Reading sqd file
input_fname = '/media/analogicalnexus/2568212B752CDB3B/MEG_Data/R2504_WMD-Filtered-raw.fif'
raw=mne.io.read_raw_fif(input_fname)
raw.load_data()
# raw.plot()

Opening raw data file /media/analogicalnexus/2568212B752CDB3B/MEG_Data/R2504_WMD-Filtered-raw.fif...
    Range : 0 ... 2551999 =      0.000 ...  2551.999 secs
Ready.
Current compensation grade : 0
Reading 0 ... 2551999  =      0.000 ...  2551.999 secs...


<Raw  |  R2504_WMD-Filtered-raw.fif, n_channels x n_times : 193 x 2552000 (2552.0 sec), ~3.67 GB, data loaded>

In [4]:
#Band pass filter
raw_40_ica = raw.filter(l_freq=0.1, h_freq=60.0)
raw_40 = raw.filter(l_freq=0.1, h_freq=60.0)
# raw_40.plot()

Setting up band-pass filter from 0.1 - 60 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 15.0 Hz
Filter length of 66000 samples (66.000 sec) selected
Setting up band-pass filter from 0.1 - 60 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 0.1 Hz
h_trans_bandwidth chosen to be 15.0 Hz
Filter length of 66000 samples (66.000 sec) selected


In [ ]:
# # compute ICA
# ica = FastICA(n_components=150)
# S_ = ica.fit_transform(raw_40)  # Get the estimated sources
# A_ = ica.mixing_  # Get estimated mixing matrix

In [5]:
events = mne.find_events(raw_40)
print(events)

459 events found
Events id: [163 164 165 173 174 175 178 187 188]
[[ 183799       0     178]
 [ 189384       0     163]
 [ 194956       0     187]
 ...
 [2508957       0     178]
 [2514520       0     175]
 [2520082       0     188]]


In [7]:
event_id = dict(r=187, nr=188)  
tmin = -.2 
tmax = 0.8
baseline = (None,0) 
picks = mne.pick_types(raw_40.info, meg=True) 
print(picks)


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156]


In [8]:
#Reference - https://martinos.org/mne/stable/auto_tutorials/plot_artifacts_correction_ica.html
#ICA parameters
n_components = 157  # if float, select n_components by explained variance of PCA
method = 'fastica'  # for comparison with EEGLAB try "extended-infomax" here
decim = 3  # we need sufficient statistics, not all time points -> saves time

# we will also set state of the random number generator - ICA is a
# non-deterministic algorithm, but we want to have the same decomposition
# and the same order of components each time this tutorial is run
random_state = 23

In [ ]:
ica = ICA(n_components=n_components, method=method, random_state=random_state)
# print(ica)
reject = dict(mag=5e-12, grad=4000e-13)
ica.fit(raw_40_ica, picks=picks, decim=decim, reject=reject)
print(ica)
ica.plot_components()

Fitting ICA to data using 157 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
    Rejecting  epoch based on MAG : [u'MEG 029', u'MEG 072', u'MEG 104', u'MEG 114', u'MEG 136', u'MEG 137', u'MEG 140', u'MEG 156']
Artifact detected in [11339, 12006]
    Rejecting  epoch based on MAG : [u'MEG 029', u'MEG 072', u'MEG 104', u'MEG 114', u'MEG 136', u'MEG 137', u'MEG 140', u'MEG 156']
Artifact detected in [13340, 14007]
    Rejecting  epoch based on MAG : [u'MEG 025', u'MEG 042', u'MEG 062']
Artifact detected in [32683, 33350]
    Rejecting  epoch based on MAG : [u'MEG 062']
Artifact detected in [33350, 34017]
    Rejecting  epoch based on MAG : [u'MEG 042', u'MEG 062']
Artifact detected in [34017, 34684]
    Rejecting  epoch based on MAG : [u'MEG 042', u'MEG 062']
Artifact detected in [34684, 35351]
    Rejecting  epoch based on MAG : [u'MEG 029', u'MEG 072', u'MEG 098', u'MEG 104', u'MEG 114', u'MEG 121', u'MEG 137', u'MEG 140', u'MEG 147', u'ME

Artifact detected in [170752, 171419]
    Rejecting  epoch based on MAG : [u'MEG 072']
Artifact detected in [171419, 172086]
    Rejecting  epoch based on MAG : [u'MEG 006', u'MEG 012', u'MEG 021', u'MEG 023', u'MEG 028', u'MEG 029', u'MEG 040', u'MEG 043', u'MEG 049', u'MEG 054', u'MEG 055', u'MEG 057', u'MEG 065', u'MEG 072', u'MEG 093', u'MEG 096', u'MEG 097', u'MEG 098', u'MEG 099', u'MEG 102', u'MEG 104', u'MEG 106', u'MEG 114', u'MEG 121', u'MEG 124', u'MEG 125', u'MEG 136', u'MEG 137', u'MEG 140', u'MEG 143', u'MEG 147', u'MEG 148', u'MEG 156']
Artifact detected in [172753, 173420]
    Rejecting  epoch based on MAG : [u'MEG 029', u'MEG 040', u'MEG 049', u'MEG 054', u'MEG 055', u'MEG 072', u'MEG 093', u'MEG 098', u'MEG 104', u'MEG 114', u'MEG 121', u'MEG 124', u'MEG 136', u'MEG 137', u'MEG 140', u'MEG 147', u'MEG 148', u'MEG 156']
Artifact detected in [174087, 174754]
    Rejecting  epoch based on MAG : [u'MEG 029', u'MEG 072', u'MEG 104', u'MEG 114', u'MEG 137', u'MEG 140', u'ME

    Rejecting  epoch based on MAG : [u'MEG 072', u'MEG 104', u'MEG 137', u'MEG 156']
Artifact detected in [692346, 693013]
    Rejecting  epoch based on MAG : [u'MEG 029', u'MEG 040', u'MEG 043', u'MEG 044', u'MEG 054', u'MEG 055', u'MEG 072', u'MEG 104', u'MEG 106', u'MEG 124', u'MEG 136', u'MEG 137', u'MEG 140', u'MEG 147', u'MEG 153', u'MEG 156']
Artifact detected in [693680, 694347]
    Rejecting  epoch based on MAG : [u'MEG 029', u'MEG 040', u'MEG 043', u'MEG 044', u'MEG 054', u'MEG 055', u'MEG 072', u'MEG 104', u'MEG 124', u'MEG 136', u'MEG 137', u'MEG 147', u'MEG 156']
Artifact detected in [695681, 696348]
    Rejecting  epoch based on MAG : [u'MEG 095']
Artifact detected in [696348, 697015]
    Rejecting  epoch based on MAG : [u'MEG 040', u'MEG 043', u'MEG 044', u'MEG 054', u'MEG 055', u'MEG 072', u'MEG 104', u'MEG 136', u'MEG 137', u'MEG 147', u'MEG 156']
Artifact detected in [697682, 698349]
    Rejecting  epoch based on MAG : [u'MEG 043', u'MEG 044', u'MEG 049', u'MEG 054', 

In [ ]:
epochs = mne.Epochs(raw_40, events, event_id, tmin, tmax, proj=False, picks=picks, baseline=baseline) 
epochs_ica = mne.Epochs(raw_40_ica, events, event_id, tmin, tmax, proj=False, picks=picks, baseline=baseline) 
evoked1 = epochs['tone1'].average()
evoked2 = epochs_ica['tone1'].average()
evoked1.plot()
evoked2.plot()

In [ ]:
evoked1.plot(spatial_colors=True, gfp=True, ylim=dict(mag=[-300,300]))
evoked2.plot(spatial_colors=True, gfp=True, ylim=dict(mag=[-300,300]))
evoked1.plot_topomap(times=[.05, .10, .15],vmin=-300,vmax=300)
evoked2.plot_topomap(times=[.05, .10, .15],vmin=-300,vmax=300)
evoked_dict = dict() 
evoked_dict['low'] = evoked1
evoked_dict['high'] = evoked2
colors=dict(low="Crimson",high="CornFlowerBlue") 
mne.viz.plot_compare_evokeds(evoked_dict, colors=colors,
picks=picks, gfp=True)

In [ ]:

la=[0,1,2,3,39,41,42,43,44,52,58,67,71,80,82,83,84,85,108,130,131,132,133,134,135,136,151]
lp=[4,5,6,7,8,9,34,36,37,38,40,45,46,47,48,49,50,75,76,77,79,87,88,90,127,129,137]
ra=[20,22,23,24,26,59,60,61,62,63,65,89,92,95,99,100,114,115,116,117,118,145,147,148,152,155]
rp=[14,15,16,17,18,19,25,27,28,30,53,54,56,57,66,68,69,70,94,96,97,119,121,122,143,144]
lh=[0,1,2,3,39,41,42,43,44,52,58,67,71,80,82,83,84,85,108,130,131,132,133,134,135,136,151,4,5,6,7,8,9,34,36,37,38,40,45,46,47,48,49,50,75,76,77,79,87,88,90,127,129,137]
rh=[20,22,23,24,26,59,60,61,62,63,65,89,92,95,99,100,114,115,116,117,118,145,147,148,152,155, 14,15,16,17,18,19,25,27,28,30,53,54,56,57,66,68,69,70,94,96,97,119,121,122,143,144]
mne.viz.plot_compare_evokeds(evoked_dict, colors=colors,
picks=lh, gfp=True, ylim=dict(mag=[0,100]))